# Solar Panel Detection from Satellite Images
This notebook demonstrates how to detect solar panels in satellite images using deep learning.

In [ ]:
# Install required packages
!pip install tensorflow opencv-python-headless numpy matplotlib

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab import files

In [ ]:
def create_model():
    # Use ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Add custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

In [ ]:
def preprocess_image(image_path):
    # Read and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize
    return img

In [ ]:
# Create model
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Upload and process image
print("Please upload a satellite image")
uploaded = files.upload()

for filename in uploaded.keys():
    # Process image
    img = preprocess_image(filename)
    
    # Make prediction
    prediction = model.predict(np.expand_dims(img, axis=0))[0][0]
    
    # Display results
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Input Image')
    
    plt.subplot(1, 2, 2)
    plt.text(0.5, 0.5, f'Solar Panel Probability: {prediction:.2%}', 
             horizontalalignment='center', verticalalignment='center')
    plt.axis('off')
    plt.show()